In [1]:
import goatools
import pandas as pd
import numpy as np
import scipy.stats as stats
import pubchempy

In [2]:
goatools

<module 'goatools' from '/usr/local/lib/python3.6/dist-packages/goatools/__init__.py'>

In [3]:
import pickle

In [4]:
drugs = ['CID3883','CID4594','CID4679']

In [5]:
with open('../data/[91.0, 88.0]-[84.0, 84.0]-[28.0, 28.0]-1.0.pkl','rb') as f:
    data = pickle.load(f)

In [6]:
with open('/home/laurence/git/posepath/PoSePath/src/tipexp_data.pkl','rb') as f:
    mapping = pickle.load(f)

In [9]:
idx1 = data['pp_idx'][0]
idx2 = data['pp_idx'][1]

In [10]:
data['pp_id'] = [[int(mapping.prot_idx_to_id[idx].replace('GeneID','')) for idx in idx1],
                 [int(mapping.prot_idx_to_id[idx].replace('GeneID','')) for idx in idx2]]

### GO enrichment

In [12]:
# Get http://geneontology.org/ontology/go-basic.obo
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

  EXISTS: go-basic.obo


In [13]:
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
from goatools.base import download_ncbi_associations
fin_gene2go = download_ncbi_associations()

  EXISTS: gene2go


In [14]:
from goatools.obo_parser import GODag

obodag = GODag("go-basic.obo")

go-basic.obo: fmt(1.2) rel(2020-01-01) 47,337 GO Terms


In [15]:
from __future__ import print_function
from goatools.anno.genetogo_reader import Gene2GoReader

# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[9606])

# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

for nspc, id2gos in ns2assoc.items():
    print("{NS} {N:,} annotated human genes".format(NS=nspc, N=len(id2gos)))

HMS:0:00:03.005510 323,107 annotations READ: gene2go 
1 taxids stored: 9606
CC 18,648 annotated human genes
MF 17,384 annotated human genes
BP 17,541 annotated human genes


In [16]:
from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT as GeneID2nt_hum

In [17]:
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS

goeaobj = GOEnrichmentStudyNS(
        GeneID2nt_hum.keys(), # List of human protein-acoding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = False,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # defult multipletest correction method


Load BP Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 80% 16,711 of 20,913 population items found in association

Load CC Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 85% 17,755 of 20,913 population items found in association

Load MF Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
 80% 16,699 of 20,913 population items found in association


In [19]:
# 'p_' means "pvalue". 'fdr_bh' is the multipletest method we are currently using.
geneids_study = data['pp_id'][0] + data['pp_id'][1] # geneid2symbol.keys()
goea_results_all = goeaobj.run_study(geneids_study)
goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]


Run BP Gene Ontology Analysis: current study set of 28 IDs ...
100%     16 of     16 study items found in association
 57%     16 of     28 study items found in population(20913)
Calculating 12,189 uncorrected p-values using fisher_scipy_stats
  12,189 GO terms are associated with 16,711 of 20,913 population items
      96 GO terms are associated with     16 of     16 study items
  METHOD fdr_bh:
      11 GO terms found significant (< 0.05=alpha) ( 11 enriched +   0 purified): statsmodels fdr_bh
       6 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)

Run CC Gene Ontology Analysis: current study set of 28 IDs ...
100%     16 of     16 study items found in association
 57%     16 of     28 study items found in population(20913)
Calculating 1,731 uncorrected p-values using fisher_scipy_stats
   1,731 GO terms are associated with 17,755 of 20,913 population items
      29 GO terms are associated with     16 of 

In [20]:
gg = goea_results_sig[0]
gg.GO

'GO:0017144'

https://github.com/tanghaibao/goatools/blob/master/notebooks/goea_nbt3102.ipynb

In [21]:
from goatools.godag_plot import plot_gos, plot_results, plot_goid2goobj


# Plot subset starting from these significant GO terms
goid_subset = [g.GO for g in goea_results_sig] # [
#     'GO:0003723', # MF D04 RNA binding (32 genes)
#     'GO:0044822', # MF D05 poly(A) RNA binding (86 genes)
#     'GO:0003729', # MF D06 mRNA binding (11 genes)
#     'GO:0019843', # MF D05 rRNA binding (6 genes)
#     'GO:0003746', # MF D06 translation elongation factor activity (5 genes)
# ]
plot_gos("go_enrich.png", 
    goid_subset, # Source GO ids
    obodag, 
    goea_results=goea_results_all) # Use pvals for coloring
# plot_results("nbt3102_BP.png", goea_results_sig)

   16 usr  52 GOs  WROTE: go_enrich.png


In [22]:
geneid2symbol = {v.GeneID: v.Symbol for k,v in GeneID2nt_hum.items()}

This plot contains GOEA results:

- GO terms colored by P-value:
    - pval < 0.005 (light red)
    - pval < 0.01 (light orange)
    - pval < 0.05 (yellow)
    - pval > 0.05 (grey) Study terms that are not statistically significant
- GO terms with study gene counts printed. e.g., "32 genes"

In [64]:
plot_gos("go_enrich_symbols.pdf", 
    goid_subset, # Source GO ids
    obodag,
    goea_results=goea_results_all, # use pvals for coloring
    # We can further configure the plot...
    id2symbol=geneid2symbol, # Print study gene Symbols, not Entrez GeneIDs
    study_items=6, # Only only 6 gene Symbols max on GO terms
    items_p_line=3, # Print 3 genes per line
    )

   16 usr  52 GOs  WROTE: go_enrich_symbols.pdf


In [24]:
from pubchempy import Compound

In [25]:
c = Compound.from_cid(5090)
c.iupac_name

'3-(4-methylsulfonylphenyl)-4-phenyl-2H-furan-5-one'

In [53]:
cids = [int(mapping.drug_idx_to_id[c].replace('CID','')) for c in data['pd_idx'][1]]
drugs = [Compound.from_cid(int(cid)) for cid in set(cids)]

In [62]:
drugs[0].cid

4594

In [55]:
drug_ids = [(d.synonyms[0], d.iupac_name) for d in drugs]
drug_ids

[('omeprazole',
  '6-methoxy-2-[(4-methoxy-3,5-dimethylpyridin-2-yl)methylsulfinyl]-1H-benzimidazole'),
 ('lansoprazole',
  '2-[[3-methyl-4-(2,2,2-trifluoroethoxy)pyridin-2-yl]methylsulfinyl]-1H-benzimidazole'),
 ('pantoprazole',
  '6-(difluoromethoxy)-2-[(3,4-dimethoxypyridin-2-yl)methylsulfinyl]-1H-benzimidazole')]

In [28]:
fps = [d.fingerprint for d in drugs]

In [63]:
geneid2symbol[5243]

'ABCB1'

import binascii
bs = binascii.unhexlify(fps[0])
bs

from scipy.spatial import distance

bfps = [binascii.unhexlify(f) for f in fps]
# distance.rogerstanimoto(bfps[0],bfps[2])

bb = bfps[0]
bb.

bfps[0]

bfps[2]

# Data will be stored in this variable
import os
geneid2symbol = {}
# Get xlsx filename where data is stored
ROOT = os.path.dirname(os.getcwd()) # go up 1 level from current working directory
din_xlsx = "../data/nbt.3102-S4_GeneIDs.xlsx"
# Read data
if os.path.isfile(din_xlsx):  
    import xlrd
    book = xlrd.open_workbook(din_xlsx)
    pg = book.sheet_by_index(0)
    for r in range(pg.nrows):
        symbol, geneid, pval = [pg.cell_value(r, c) for c in range(pg.ncols)]
        if geneid:
            geneid2symbol[int(geneid)] = symbol
    print('{N} genes READ: {XLSX}'.format(N=len(geneid2symbol), XLSX=din_xlsx))
else:
    raise RuntimeError('FILE NOT FOUND: {XLSX}'.format(XLSX=din_xlsx))